In [47]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select

driver = webdriver.Chrome()
driver.get("https://www.timeanddate.com/weather/usa/palo-alto/historic")


In [48]:
import pandas as pd
from time import sleep

start_month=9
start_day=24
end_month=10
end_day=15

directory = "/Users/vincewu/Bees-Project"
processed_folder = "Fully Processed"

months_num = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

arr=[]

for month in range(start_month, end_month+1):
    if month < 10:
        month_str = '0'+str(month)
    else:
        month_str = str(month)
    month_select = Select(driver.find_element('id', 'month'))
    month_select.select_by_value('2022-'+month_str)

    print('2022-'+month_str)

    first_day=1
    last_day=months_num[month-1]
    if month==start_month: first_day=start_day
    if month==end_month: last_day=end_day

    #print('Month: ', month)
    print(first_day, last_day)
    for day in range(first_day, last_day+1):
        # print('Day: ', day)
        #if day!=1:
        if day < 10:
            day_str = '0'+str(day)
        else:
            day_str = str(day)

        day_select = Select(driver.find_element('id', 'wt-his-select'))
        day_select.select_by_value('2022'+month_str+day_str)
        print('2022'+month_str+day_str)
        
        sleep(0.1)

        table = driver.find_element('id', 'wt-his')
        table_text = table.text
        split_table = table_text.split()
        # print(split_table)

        #discard 9 from top, 5 from bottom
        #7 columns equates to 14 individual space-separated strings
        #subtract 3 for the first row having "day, month day"

        n = 0
        for cell in split_table:
            if cell == 'am' or cell == 'pm':
                n += 1

        index = 9
        # n=(len(split_table)-14+9-3)/14
        for i in range(n):
            arr.append([None for i in range(13)])
            arr[-1][0]=month
            arr[-1][1] = day
            # arr[-1][0]=split_table[12]
            # arr[-1][1] = split_table[13]
            while True:
                cell = split_table[index]
                index += 1
                if cell[0].isdigit():
                    #time
                    if split_table[index] == 'am' or split_table[index] == 'pm':
                        arr[-1][2] = cell
                    elif split_table[index] == '°F':
                        arr[-1][7] = cell
                    #air temperature
                    elif split_table[index] == 'mph':
                        arr[-1][3] = 'x '+cell
                    #humidity
                    elif split_table[index][0].isdigit():
                        arr[-1][9] = cell
                    #pressure
                    elif split_table[index] == '"Hg':
                        arr[-1][12] = cell
                    #visibility
                    elif split_table[index] == 'mi':
                        arr[-1][4] = cell
                        break




2022-09
24 30
20220924
20220925
20220926
20220927
20220928
20220929
20220930
2022-10
1 15
20221001
20221002
20221003
20221004
20221005
20221006
20221007
20221008
20221009
20221010
20221011
20221012
20221013
20221014
20221015


In [49]:
arr

[[9,
  24,
  '6:47',
  None,
  '10',
  None,
  None,
  '55',
  None,
  '88%',
  None,
  None,
  '29.99'],
 [9,
  24,
  '9:47',
  None,
  '10',
  None,
  None,
  '70',
  None,
  '69%',
  None,
  None,
  '30.04'],
 [9,
  24,
  '10:47',
  'x 5',
  '10',
  None,
  None,
  '72',
  None,
  '61%',
  None,
  None,
  '30.05'],
 [9,
  24,
  '11:47',
  'x 6',
  '10',
  None,
  None,
  '75',
  None,
  '54%',
  None,
  None,
  '30.04'],
 [9,
  24,
  '12:47',
  'x 8',
  '10',
  None,
  None,
  '75',
  None,
  '69%',
  None,
  None,
  '30.02'],
 [9,
  24,
  '1:47',
  'x 9',
  '10',
  None,
  None,
  '77',
  None,
  '61%',
  None,
  None,
  '30.01'],
 [9,
  24,
  '2:47',
  'x 12',
  '10',
  None,
  None,
  '82',
  None,
  '48%',
  None,
  None,
  '29.99'],
 [9,
  24,
  '3:47',
  'x 13',
  '10',
  None,
  None,
  '81',
  None,
  '51%',
  None,
  None,
  '29.98'],
 [9,
  24,
  '5:47',
  'x 14',
  '10',
  None,
  None,
  '75',
  None,
  '65%',
  None,
  None,
  '29.97'],
 [9,
  24,
  '6:47',
  'x 13',
  

In [51]:

df = pd.DataFrame(arr)
df = df.loc[::-1]
columns = ['Month', 'Day', 'Time', 'Wind (mph)',
           'Visibility (miles)', 'Weather', 'Sky Condition',
           'Air Temperature', 'Dew Point', 'Relative Humidity',
           'Wind Chill', 'Heat Index', 'Altimeter (inHg)']
df.to_csv(directory+'/'+processed_folder+'/weather_temp.csv', header=columns)
